In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import gym 
import random
import tensorflow as tf
import numpy as np

# Testing Random Enviroment in OpenAI Gym

In [3]:
env = gym.make('MsPacman-v0')

In [4]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [5]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5,6,7,8])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} score:{}'.format(episode, score))
env.close()

Episode:1 score:210.0
Episode:2 score:220.0
Episode:3 score:330.0
Episode:4 score:160.0
Episode:5 score:210.0


# Creating a Deep Learning Model with Keras

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(128, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [9]:
model = build_model(height, width, channels, actions)

In [16]:
del model

In [17]:
model = build_model(height, width, channels, actions)

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 128)    24704     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     131136    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 11, 8, 64)      65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 16896)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               8651264   
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 9)                

# Build an Agent with keras

In [13]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [14]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
del model

In [ ]:
model = build_model(height, width, channels, actions)

In [18]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [19]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


C:\Users\SAMEER\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


  569/10000: episode: 1, duration: 15.051s, episode steps: 569, steps per second:  38, episode reward: 110.000, mean reward:  0.193 [ 0.000, 10.000], mean action: 3.919 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\SAMEER\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1260/10000: episode: 2, duration: 548.891s, episode steps: 691, steps per second:   1, episode reward: 190.000, mean reward:  0.275 [ 0.000, 10.000], mean action: 3.767 [0.000, 8.000],  loss: 22.684262, mean_q: 5.154851, mean_eps: 0.898300
 1837/10000: episode: 3, duration: 1208.125s, episode steps: 577, steps per second:   0, episode reward: 190.000, mean reward:  0.329 [ 0.000, 10.000], mean action: 3.943 [0.000, 8.000],  loss: 1.042983, mean_q: 3.873156, mean_eps: 0.860680
 2494/10000: episode: 4, duration: 1416.141s, episode steps: 657, steps per second:   0, episode reward: 210.000, mean reward:  0.320 [ 0.000, 10.000], mean action: 4.139 [0.000, 8.000],  loss: 0.689169, mean_q: 3.032516, mean_eps: 0.805150
 3144/10000: episode: 5, duration: 1401.674s, episode steps: 650, steps per second:   0, episode reward: 220.000, mean reward:  0.338 [ 0.000, 10.000], mean action: 4.285 [0.000, 8.000],  loss: 0.555421, mean_q: 2.883585, mean_eps: 0.746335
 3997/10000: episode: 6, duration: 1

In [1]:
scores = dqn.test(env, nb_episodes=5, visualize=True, verbose=2)
print(np.mean(scores.history['episode_reward']))

NameError: name 'dqn' is not defined

# Saving the model 

In [1]:
dqn.save_weights('dqn_weights.h5f')

NameError: name 'dqn' is not defined

In [22]:
scores